# Setup

In [ ]:
import openai
from dotenv import load_dotenv
import os
import json

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Ejemplo de llamada y respuesta

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {
          "role": "system",
          "content": """
Eres un analista de sentimientos.
Dada una oración, debes analizar si la oración tiene un sentimiento positivo o negativo.
Debes responder solamente con "positivo" o "negativo" al mensaje del usuario.
El mensaje del usuario es la oración que debes catalogar.
          """
        },
        {"role": "user", "content": "Hola, estoy super contento hoy!"},
    ]
)

print(response.choices[0].message)

# Definir las funciones que queremos proporcionar al modelo

In [ ]:
todo_list = []

def add_to_list(item):
    todo_list.append(item)
    return "Agregado a tu lista: " + item

def get_list():
    if (len(todo_list) == 0):
        return "Tu lista está vacía"
    else:
        return 'Tu lista actual es: \n - ' + '\n - '.join(todo_list)

def remove_from_list(item):
    if item in todo_list:
        todo_list.remove(item)
        return "Quitado de tu lista: " + item
    else:
        return "Item no encontrado"

# Definir el JSON Schema

In [ ]:
functions_schema = [
    {
        "name": "add_to_list",
        "description": "Agrega un item a la lista de tareas",
        "parameters": {
            "type": "object",
            "properties": {
                "item": {
                    "type": "string",
                    "description": "El item a agregar a la lista"
                }
            },
            "required": ["item"]
        },
    },
    {
        "name": "get_list",
        "description": "Obtiene la lista de tareas",
        "parameters": {
            "type": "object",
            "properties": {}
        },
    },
    {
        "name": "remove_from_list",
        "description": "Quita un item de la lista de tareas",
        "parameters": {
            "type": "object",
            "properties": {
                "item": {
                    "type": "string",
                    "description": "El item a quitar de la lista"
                }
            },
            "required": ["item"]
        },
    }
]

# Ejemplo de respuesta con funciones

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    functions=functions_schema,
    function_call="auto",
    messages=[
        {
          "role": "system",
          "content": """
Eres un asistente de una app de lista de tareas.
Debes responder los mensajes del usuario y llamar a la función correspondiente si es necesario.
          """
        },
        {"role": "user", "content": "Tengo que comprar pan"},
    ]
)

print(response.choices[0].message)

# Mapear el nombre de las funciones a las funciones reales

In [ ]:
available_functions = {
    "add_to_list": add_to_list,
    "get_list": get_list,
    "remove_from_list": remove_from_list
}

def call_function(name, arguments):
    return available_functions[name](**arguments)

In [ ]:
history = []

def call_openai_with_history():
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        functions=functions_schema,
        function_call="auto",
        messages=[
            {
                "role": "system",
                "content": """
Eres un asistente de una app de lista de tareas.
Debes responder los mensajes del usuario y llamar a la función correspondiente si es necesario.
                """
            }
        ] + history
    )


def call_todo_agent(text):
    history.append({"role": "user", "content": text})

    response = call_openai_with_history()

    print(response.choices[0].message)

    if response.choices[0].message.get('function_call') != None:
        function_call = response.choices[0].message.function_call
        arguments = json.loads(function_call.arguments)
        function_response = call_function(function_call.name, arguments)

        print(function_response)
        history.append(
            {"role": "function", "name": function_call.name, "content": function_response})
    else:
        history.append(
            {"role": "assistant", "content": response.choices[0].message.content})
        print(response.choices[0].message.content)

In [ ]:
call_todo_agent("Hola")